# Исследование поведения пользователей 2018

в рамках этого исследования нужно ответить на два вопроса:

1. Зависит ли вероятность оплаты от выбранного пользователем уровня сложности? (есть ли зависимость между выбранным уровнем сложности и вероятностью оплаты)
2. Существует ли разница во времени между событиями регистрации и оплаты для разных групп пользователей с разным уровнем сложности? (различается ли временной промежуток между регистрацией и оплатой у групп пользователей с разным уровнем сложности)

In [2]:
import pandas as pd
import numpy as np

purchase = pd.read_csv(r'C:\Users\Yana\Documents\purchase.csv', sep=',')
events = pd.read_csv(r'C:\Users\Yana\Documents\7_4_Events.csv', sep=',')

# purchase = pd.read_csv('purchase.csv') 
# events = pd.read_csv('7_4_Events.csv')

In [3]:
# создать фильтр 2018
cond2018 = (events.start_time>='2018-01-01') & (events.start_time<'2019-01-01') & (events.event_type=='registration')
registered = events[cond2018]  
# список пользователей, зарег. в 2018
lst_reg = registered['user_id'].to_list() 
# отфильтровать пользователей зарег в 2018
events_df = events[events.user_id.isin(lst_reg)].copy() 
purchase_df = purchase[purchase.user_id.isin(lst_reg)].copy() 
# добавить ивент тип покупка
purchase_df['event_type'] = 'purchase' 
# изменить тип 
purchase_df['event_datetime'] = pd.to_datetime(purchase_df['event_datetime'])
# переименовать айди
events_df = events_df.rename(columns={"id": "event_id"}) 
purchase_df = purchase_df.rename(columns={"id": "purchase_id"})
# объединить датафреймы
total_events_df = pd.concat([events_df,purchase_df],sort=False) 
# изменить тип 
total_events_df['start_time'] = pd.to_datetime(total_events_df['start_time'])
#total_events_df.info()
total_events_df

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,event_datetime,amount
51405,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaT,NaN
51406,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaT,NaN
51407,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaT,NaN
51408,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaT,NaN
51409,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...
2767,NaN,purchase,NaN,NaT,NaN,47498,18441.0,2019-01-02 03:48:19,100.0
2768,NaN,purchase,NaN,NaT,NaN,47647,18442.0,2019-01-02 23:26:26,150.0
2769,NaN,purchase,NaN,NaT,NaN,47554,18443.0,2019-01-03 00:36:36,50.0
2774,NaN,purchase,NaN,NaT,NaN,47742,18448.0,2019-01-04 12:51:41,50.0


In [3]:
# проверка 
user_levels = total_events_df[~total_events_df['selected_level'].isnull()].copy()
user_levels_df = user_levels[['selected_level', 'user_id', 'start_time']]
print(user_levels_df['user_id'].nunique()) # 8342 юзеров выбрали уровень сложности
print(user_levels_df.groupby('user_id')['selected_level'].count().mean()) # один уровень на юзера

8342
1.0


Выделите группы пользователей

In [4]:
group_easy = user_levels_df[(user_levels_df['selected_level'] == 'easy')]["user_id"].unique()
group_medium = user_levels_df[(user_levels_df['selected_level'] == 'medium')]["user_id"].unique()
group_hard = user_levels_df[(user_levels_df['selected_level'] == 'hard')]["user_id"].unique()
set_easy = set(group_easy) #список выбравших низкий уровень сложности
set_medium = set(group_medium) #список выбравших средний уровень сложности
set_hard = set(group_hard) #список выбравших высокий уровень сложности
print(len(set_easy)+len(set_medium)+len(set_hard)) # совпадает с ранее рассчитанным количеством

8342


# UPDATED CALCULATIONS

In [5]:
# переименовать колонку event_datetime на start_time в таблице purchase_df чтобы подготовить к объединению 
purchase_df_2 = purchase_df.copy().rename(columns={"event_datetime": "start_time"})
total_events_2 = pd.concat([events_df,purchase_df_2],sort=False) # объединить датафреймы

# здесь можем суммировать даты потому что для всех событый кроме прохождения туториала есть только ОДНА дата
a = total_events_2.pivot_table(
    values='start_time',
    index= ['user_id'],
    columns= 'event_type',
    aggfunc='sum')

# вытаскиваем user_id из индекса в колонку
a.reset_index(level=0, inplace=True)
# убираем лишние колонки
del a["tutorial_finish"]
del a["tutorial_start"]
del a["pack_choice"] 
# меняем тип на дату
a['registration'] = pd.to_datetime(a['registration'])
a['level_choice'] = pd.to_datetime(a['level_choice'])
display(a)

# расчет времени между событиями для каждого пользователя
a['timedelta'] = a['purchase'] - a['registration'] #разница между регистрацией и оплатой
a['timedelta2'] = a['level_choice'] - a['registration'] #разница между регистрацией и выбором уровня
a['timedelta3'] = a['purchase'] - a['level_choice'] #разница между выбором уровня и оплатой

event_type,user_id,level_choice,purchase,registration
0,27832,NaT,NaT,2018-01-01 03:48:40
1,27833,NaT,NaT,2018-01-01 04:07:25
2,27834,NaT,NaT,2018-01-01 08:35:10
3,27835,2018-01-01 20:37:22,NaT,2018-01-01 11:54:47
4,27836,NaT,NaT,2018-01-01 13:28:07
...,...,...,...,...
19921,47753,2018-12-31 22:36:19,NaT,2018-12-31 18:58:55
19922,47754,NaT,NaT,2018-12-31 19:14:08
19923,47755,2019-01-01 05:04:52,NaT,2018-12-31 21:15:14
19924,47756,2019-01-01 05:42:11,NaT,2018-12-31 23:17:30


In [6]:
# расчет среднего времени между событиями для пользователей выбравших низкий уровень сложности
easy_df = a[a["user_id"].isin(set_easy)].copy()
easy_time = easy_df['timedelta'].mean() #reg to purchase
easy_time2 = easy_df['timedelta2'].mean() #reg to select level
easy_time3 = easy_df['timedelta3'].mean() #select level to purchase
easy_total = easy_df['user_id'].nunique()
easy_buyers = easy_df[~easy_df['purchase'].isnull()]['user_id'].nunique()
easy_percent = round((easy_buyers/easy_total*100),2)

# расчет среднего времени между событиями для пользователей выбравших средний уровень сложности
medium_df = a[a["user_id"].isin(set_medium)].copy()
medium_time = medium_df['timedelta'].mean() #reg to purchase
medium_time2 = medium_df['timedelta2'].mean() #reg to select level
medium_time3 = medium_df['timedelta3'].mean() #select level to purchase
medium_total = medium_df['user_id'].nunique()
medium_buyers = medium_df[~medium_df['purchase'].isnull()]['user_id'].nunique()
medium_percent = round((medium_buyers/medium_total*100),2)

# расчет среднего времени между событиями для пользователей выбравших высокий уровень сложности
hard_df = a[a["user_id"].isin(set_hard)].copy()
hard_time = hard_df['timedelta'].mean() #reg to purchase
hard_time2 = hard_df['timedelta2'].mean() #reg to select level
hard_time3 = hard_df['timedelta3'].mean() #select level to purchase
hard_total = hard_df['user_id'].nunique()
hard_buyers = hard_df[~hard_df['purchase'].isnull()]['user_id'].nunique()
hard_percent = round((hard_buyers/hard_total*100),2)

In [7]:
# вывод данных
# columns
total = [easy_total, medium_total, hard_total]
buyers = [easy_buyers, medium_buyers, hard_buyers]
percent = [easy_percent, medium_percent, hard_percent]
timedelta_reg_pur = [easy_time, medium_time, hard_time]
timedelta_reg_lvl = [easy_time2, medium_time2, hard_time2]
timedelta_lvl_pur = [easy_time3, medium_time3, hard_time3]
  
# index
selected_level = ['easy', 'medium', 'hard']
  
# полная таблица с результатами
result_df = pd.DataFrame(list(zip(selected_level, total, buyers, percent, 
                                  timedelta_reg_pur, timedelta_reg_lvl, timedelta_lvl_pur)),
                  columns = ['selected_level','total','buyers','percent',
                             'timedelta_reg_purchase','timedelta_reg_lvl','timedelta_lvl_purchase'])
result_df

,selected_level,total,buyers,percent,timedelta_reg_purchase,timedelta_reg_lvl,timedelta_lvl_purchase
0,easy,2448,189,7.72,3 days 22:10:23.211640211,0 days 07:10:06.324754901,3 days 14:58:52.941798941
1,medium,4645,969,20.86,4 days 06:12:06.576883384,0 days 07:08:13.360387513,3 days 23:14:13.165118679
2,hard,1249,442,35.39,3 days 14:55:19.257918552,0 days 07:18:32.228182546,3 days 07:20:41.420814479


Есть ли зависимость между выбранным уровнем сложности и вероятностью оплаты?

1. наибольшее количество пользователей (4645 человек) выбрало средний уровень сложности, каждый 5й совершал покупку, но им в среднем понадобилось больше всего времени (почти 4 дня) чтобы совершить на покупку.

2. наименьшее количество пользователей (1249 человек) выбрало высокий уровень сложности, каждый 3й совершал покупку, однако им в среднем понадобилось меньше всего времени (3 дня и 7,35 часа) чтобы совершить покупку.

3. те, кто выбрал низкий уровень сложности (примерно треть от всех, кто выбрал уровень сложности) редко совершали покупку: только 7,72 %.

**Вывод: чем выше уровень трудности выбрал пользователь, тем вероятнее что он совершит покупку.**

Существует ли разница во времени между событиями регистрации и оплаты?
- опять же, быстрее всех совершают покупку пользователи, выбравшие высокий уровень сложности (за 3 дня и 15 часов).
- медленнее всех совершают покупку пользователи, выбравшие средний уровень сложности (за 4 дня и 6,2 часов).
- те, кто выбрал низкий уровень сложности совершают покупку в среднем через 3 дня и 22,2 часов после регистрации.

похоже что время между регистрацией и выбором уровня для разных групп различается мало, и в среднем составляет 7 часов и 12 минут